In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import tetrion.plots as plots
import tetrion.db
import tetrion.commands as cmd
import tetrion.options
import tetrion.clientcore
import seaborn as sns
from collections import defaultdict
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
%matplotlib inline 
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
# import sys
# sys.path.append('../utils')  # Adjust the path to where the eventstudy module is located
# import eventstudy as es

In [2]:
start_date = '2025-05-01'
end_date = '2025-05-06'
date_range = pd.date_range(start = start_date, end = end_date, freq='D')


txf_books, txf_ticks = pd.DataFrame(), pd.DataFrame()
for date in date_range:#dt.date()
    try:
        txf_book = cmd.book_printer_v2('FUT_TAIFEX_TXF:202505',sdate = date, edate = date,depth=2,source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ')
        txf_tick = cmd.tick_printer('FUT_TAIFEX_TXF:202505',sdate = date, edate = date)
        
        txf_books = pd.concat([txf_books,txf_book], axis = 0)
        txf_ticks = pd.concat([txf_ticks,txf_tick], axis = 0)
    except:
        pass

missing datafile SRC_TAIFEX.ORIG_CPBQ.DEST_CPBQ FUT_TAIFEX_TXF 20250504 file=/nfs/datafiles/2025/05/04/59709/datafile.944111726.gz


In [51]:
txf_ticks_clean = txf_ticks.drop(columns=['bid_px', 'ask_px'])

In [52]:
merged_df = pd.merge_asof(
    txf_ticks_clean.sort_values('time'),
    txf_books.sort_values('time'),
    on='time',
    direction='backward',  # 成交點對上最靠近的過去一筆 book
    tolerance=pd.Timedelta(milliseconds=500),  # 最多允許相差 500 毫秒
)

In [53]:
traded = merged_df[merged_df['sz'] != 0].copy()

In [54]:
merged_df

,time,ex_time,instr,sz,px,bid_sz_x,ask_sz_x,bid_sz4,bid_sz3,bid_sz2,bid_sz1,bid_sz_y,bid_px4,bid_px3,bid_px2,bid_px1,bid_px,mid_px,ask_px,ask_px1,ask_px2,ask_px3,ask_px4,ask_sz_y,ask_sz1,ask_sz2,ask_sz3,ask_sz4
0,2025-05-01 00:00:00.103446,2025-05-01 00:00:00.095,FUT_TAIFEX_TXF:202505,1,20088.0,0,0,NaN,14.0,10.0,10.0,8.0,NaN,20082.0,20083.0,20085.0,20086.0,20087.5,20089.0,20090.0,20091.0,20093.0,NaN,8.0,8.0,8.0,7.0,NaN
1,2025-05-01 00:00:00.103459,2025-05-01 00:00:00.095,FUT_TAIFEX_TXF:202505,1,20088.0,0,0,NaN,14.0,10.0,10.0,8.0,NaN,20082.0,20083.0,20085.0,20086.0,20087.5,20089.0,20090.0,20091.0,20093.0,NaN,8.0,8.0,8.0,7.0,NaN
2,2025-05-01 00:00:00.228509,2025-05-01 00:00:00.190,FUT_TAIFEX_TXF:202505,1,20089.0,8,8,0.0,14.0,10.0,10.0,3.0,0.0,20082.0,20083.0,20085.0,20086.0,20087.0,20088.0,20089.0,20090.0,20091.0,20093.0,2.0,9.0,8.0,9.0,7.0
3,2025-05-01 00:00:00.228547,2025-05-01 00:00:00.217,FUT_TAIFEX_TXF:202505,1,20088.0,8,8,0.0,14.0,10.0,10.0,3.0,0.0,20082.0,20083.0,20085.0,20086.0,20087.0,20088.0,20089.0,20090.0,20091.0,20093.0,2.0,9.0,8.0,9.0,7.0
4,2025-05-01 00:00:00.353471,2025-05-01 00:00:00.235,FUT_TAIFEX_TXF:202505,1,20088.0,3,2,0.0,14.0,10.0,10.0,5.0,0.0,20082.0,20083.0,20085.0,20086.0,20087.0,20088.0,20089.0,20090.0,20091.0,20093.0,1.0,9.0,8.0,9.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225267,2025-05-06 23:59:57.434721,2025-05-06 23:59:57.368,FUT_TAIFEX_TXF:202505,1,20312.0,2,4,10.0,9.0,9.0,9.0,5.0,20307.0,20308.0,20309.0,20310.0,20311.0,20312.0,20313.0,20314.0,20315.0,20316.0,20317.0,10.0,10.0,10.0,13.0,10.0
225268,2025-05-06 23:59:57.434743,2025-05-06 23:59:57.368,FUT_TAIFEX_TXF:202505,1,20312.0,2,4,10.0,9.0,9.0,9.0,5.0,20307.0,20308.0,20309.0,20310.0,20311.0,20312.0,20313.0,20314.0,20315.0,20316.0,20317.0,10.0,10.0,10.0,13.0,10.0
225269,2025-05-06 23:59:57.934728,2025-05-06 23:59:57.898,FUT_TAIFEX_TXF:202505,1,20311.0,2,14,14.0,9.0,10.0,11.0,8.0,20305.0,20306.0,20307.0,20308.0,20309.0,20310.5,20312.0,20313.0,20314.0,20315.0,20316.0,10.0,12.0,7.0,10.0,14.0
225270,2025-05-06 23:59:57.934762,2025-05-06 23:59:57.899,FUT_TAIFEX_TXF:202505,1,20311.0,2,14,14.0,9.0,10.0,11.0,8.0,20305.0,20306.0,20307.0,20308.0,20309.0,20310.5,20312.0,20313.0,20314.0,20315.0,20316.0,10.0,12.0,7.0,10.0,14.0


In [55]:
# 假設 merged_df 來自：txf_ticks 與 txf_books 合併 + ffill 過的資料
# 只篩出有成交的資料
traded = merged_df[merged_df['sz'] != 0].copy()
# 定義穿價條件
up_cross = traded['px'] > traded['ask_px']
down_cross = traded['px'] < traded['bid_px']

# 建立穿價標籤
traded['trade_through'] = 'none'
traded.loc[up_cross, 'trade_through'] = 'upward'
traded.loc[down_cross, 'trade_through'] = 'downward'

# 只留下有穿價的成交
trade_through_df = traded[traded['trade_through'] != 'none']


In [41]:
# # 定義穿價條件
# up_cross = traded['px'] > traded['ask_px4'] 
# down_cross = traded['px'] < traded['bid_px4']

# # 建立穿價標籤
# traded['trade_through'] = 'none'
# traded.loc[up_cross, 'trade_through'] = 'upward'
# traded.loc[down_cross, 'trade_through'] = 'downward'

# # 只留下有穿價的成交
# trade_through_df = traded[traded['trade_through'] != 'none']
# 篩出成交資料
traded = merged_df[merged_df['sz'] != 0].copy()

# 定義穿價條件（避免 ask_px4 或 bid_px4 為 0）
up_cross = (traded['ask_px4'] != 0) & (traded['px'] > traded['ask_px4'])
down_cross = (traded['bid_px4'] != 0) & (traded['px'] < traded['bid_px4'])

# 建立穿價標籤
traded['trade_through'] = 'none'
traded.loc[up_cross, 'trade_through'] = 'upward'
traded.loc[down_cross, 'trade_through'] = 'downward'


# 只留下有穿價的成交
trade_through_df = traded[traded['trade_through'] != 'none']


In [56]:
import pandas as pd

def group_trade_through_events(df, time_col='time', label_col='trade_through', window_seconds=1):
    """
    將穿價成交依時間聚類：1 秒內視為同一事件
    會加入欄位：'time_diff', 'event_group'
    """
    df = df.sort_values(time_col).copy()
    # df[time_col] = pd.to_datetime(df[time_col]) 
    df['time_diff'] = df[time_col].diff().dt.total_seconds().fillna(9999)
    df['event_group'] = (df['time_diff'] > window_seconds).cumsum()
    return df


def summarize_trade_through_events(trade_through_df, window_seconds=1):
    """
    對穿價資料分群，輸出每組穿價事件的代表方向與時間
    """
    # 1. 還原 index 為欄位
    if isinstance(trade_through_df.index, pd.DatetimeIndex):
        trade_through_df = trade_through_df.reset_index()  # index: time → 欄位: time

    # 2. 分群
    grouped_df = group_trade_through_events(trade_through_df, time_col='time', window_seconds=window_seconds)

    # 3. 每組只保留第一筆穿價事件
    summary = grouped_df.groupby('event_group').first()[['time', 'trade_through']].reset_index()

    return summary


In [57]:
summary_df = summarize_trade_through_events(trade_through_df, window_seconds=1)

### 市場TXF所有穿價的時間點及方向

In [58]:
summary_df

,event_group,time,trade_through
0,1,2025-05-01 00:00:00.228509,upward
1,2,2025-05-01 00:00:01.728474,upward
2,3,2025-05-01 00:00:09.978466,upward
3,4,2025-05-01 00:01:40.978439,downward
4,5,2025-05-01 00:02:21.978485,upward
...,...,...,...
6275,6276,2025-05-06 23:51:37.684727,downward
6276,6277,2025-05-06 23:52:36.809718,downward
6277,6278,2025-05-06 23:53:44.059694,upward
6278,6279,2025-05-06 23:54:06.684684,downward


### 獲取我們的成交資料

In [59]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from eddie_tools.record import RedisFillLoader

In [60]:
tmf = RedisFillLoader('2025,5,1','2025,5,6','capital_electron_tmf')

In [61]:
df = tmf.load_concat_df()

Concatenating fills: 100%|██████████| 6/6 [00:00<00:00, 16.22it/s]


In [62]:
fills_df = df[df['instr'].str.contains('tmf:202505', case=False, na=False)]

In [63]:
fills_df

,time,ts,instr,oflag,tid,px,alqortake,sz,theo,fwd,ffwd,edge,roll_offset,label,delta,vega,gamma,theta,vol,mult,latefill,fee,qm,pfdelta,otags,seq
0,2025-04-30 15:00:05.508137,1745996405508137,FUT_TAIFEX_TMF:202505,0,TAIFEX155#185,20187,alq,1,20187.5,20187.5,20187.5,0,0,,1,0,0,0,0,10,False,1.20374,0,0,0x20000,26
1,2025-04-30 15:00:05.811946,1745996405811946,FUT_TAIFEX_TMF:202505,0,TAIFEX155#255,20183,alq,2,20186.5,20186.5,20186.5,0,0,,1,0,0,0,0,10,False,1.20366,0,0,0x20000,27
2,2025-04-30 15:00:05.891461,1745996405891461,FUT_TAIFEX_TMF:202505,0,TAIFEX155#255,20183,alq,2,20186.5,20186.5,20186.5,0,0,,1,0,0,0,0,10,False,1.20366,0,0,0x20000,28
3,2025-04-30 15:00:09.305926,1745996409305926,FUT_TAIFEX_TMF:202505,0,TAIFEX155#442,20186,alq,-5,20185.5,20185.5,20185.5,0,0,,1,0,0,0,0,10,False,1.20372,0,0,0x20000,29
4,2025-04-30 19:28:14.002964,1746012494002964,FUT_TAIFEX_TMF:202505,0,TAIFEX155#167037,20211,alq,-2,20209.5,20209.5,20209.5,0,0,,1,0,0,0,0,10,False,1.20422,0,0,0x20000,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2025-05-06 09:13:39.330003,1746494019330003,FUT_TAIFEX_TMF:202505,0,TAIFEX155#136084,20251,take,4,20252.0,20252.0,20252.0,0,0,,1,0,0,0,0,10,False,1.20502,0,0,0x0,11
607,2025-05-06 09:13:39.330057,1746494019330057,FUT_TAIFEX_TMF:202505,0,TAIFEX155#136084,20252,take,5,20252.0,20252.0,20252.0,0,0,,1,0,0,0,0,10,False,1.20504,0,0,0x0,12
608,2025-05-06 09:13:39.330068,1746494019330068,FUT_TAIFEX_TMF:202505,0,TAIFEX155#136084,20253,take,2,20252.0,20252.0,20252.0,0,0,,1,0,0,0,0,10,False,1.20506,0,0,0x0,13
609,2025-05-06 09:13:39.330074,1746494019330074,FUT_TAIFEX_TMF:202505,0,TAIFEX155#136084,20254,take,5,20252.0,20252.0,20252.0,0,0,,1,0,0,0,0,10,False,1.20508,0,0,0x0,14


> 把我們的成交紀錄與市場穿價紀錄做比對 並合併

In [64]:
my_fills_df = fills_df.copy()
if isinstance(my_fills_df.index, pd.DatetimeIndex):
    my_fills_df = my_fills_df.reset_index()

my_fills_df['time'] = pd.to_datetime(my_fills_df['time'])
summary_df['time'] = pd.to_datetime(summary_df['time'])


In [65]:
# 必須先排序時間
my_fills_df = my_fills_df.sort_values('time')
summary_df = summary_df.sort_values('time')
# 往前merge
merged = pd.merge_asof(
    my_fills_df,
    summary_df,
    on='time',
    direction='backward',
    tolerance=pd.Timedelta(seconds=1)  
)


In [66]:
merged['is_trade_near_cross'] = merged['trade_through'].notna()
merged_df = merged[['time','px','sz','instr','trade_through','is_trade_near_cross']]

In [67]:
merged_df

,time,px,sz,instr,trade_through,is_trade_near_cross
0,2025-04-30 15:00:05.508137,20187,1,FUT_TAIFEX_TMF:202505,NaN,False
1,2025-04-30 15:00:05.811946,20183,2,FUT_TAIFEX_TMF:202505,NaN,False
2,2025-04-30 15:00:05.891461,20183,2,FUT_TAIFEX_TMF:202505,NaN,False
3,2025-04-30 15:00:09.305926,20186,-5,FUT_TAIFEX_TMF:202505,NaN,False
4,2025-04-30 19:28:14.002964,20211,-2,FUT_TAIFEX_TMF:202505,NaN,False
...,...,...,...,...,...,...
365,2025-05-06 09:13:39.330003,20251,4,FUT_TAIFEX_TMF:202505,NaN,False
366,2025-05-06 09:13:39.330057,20252,5,FUT_TAIFEX_TMF:202505,NaN,False
367,2025-05-06 09:13:39.330068,20253,2,FUT_TAIFEX_TMF:202505,NaN,False
368,2025-05-06 09:13:39.330074,20254,5,FUT_TAIFEX_TMF:202505,NaN,False


In [68]:
merged_df.dropna()

,time,px,sz,instr,trade_through,is_trade_near_cross
39,2025-05-01 01:55:26.058756,20150,40,FUT_TAIFEX_TMF:202505,downward,True
101,2025-05-01 04:12:37.922094,20298,1,FUT_TAIFEX_TMF:202505,upward,True
102,2025-05-01 04:12:37.922102,20300,39,FUT_TAIFEX_TMF:202505,upward,True
103,2025-05-01 04:12:37.922651,20300,11,FUT_TAIFEX_TMF:202505,upward,True
104,2025-05-01 04:12:37.922723,20301,5,FUT_TAIFEX_TMF:202505,upward,True
105,2025-05-01 04:12:37.922723,20302,7,FUT_TAIFEX_TMF:202505,upward,True
106,2025-05-01 04:12:37.922723,20303,17,FUT_TAIFEX_TMF:202505,upward,True
191,2025-05-02 08:45:00.080247,20480,9,FUT_TAIFEX_TMF:202505,upward,True
192,2025-05-02 08:45:00.080306,20481,2,FUT_TAIFEX_TMF:202505,upward,True
193,2025-05-02 08:45:00.082805,20482,3,FUT_TAIFEX_TMF:202505,upward,True


### 把同一秒內的成交看做一個事件 

In [69]:
def group_within_seconds(df, time_col='time', threshold_seconds=1):

    df = df.sort_values(time_col).copy()
    df[time_col] = pd.to_datetime(df[time_col])  # 確保是 datetime

    # 計算與前一筆的時間差（秒）
    df['time_diff'] = df[time_col].diff().dt.total_seconds().fillna(float('inf'))

    # 超過 threshold 秒就視為新群組
    df['group_id'] = (df['time_diff'] > threshold_seconds).cumsum()

    return df


In [70]:
grouped = group_within_seconds(merged_df, time_col='time', threshold_seconds=1)

# 合併同一群內的 sz，其餘欄位依需要處理
merged_sec_df = (
    grouped
    .groupby('group_id')
    .agg(
        time=('time', 'first'),                  # 群組代表時間取第一筆
        sz=('sz', 'sum'),                        # 口數相加
        trade_through=('trade_through', 'first'),# 穿價方向取第一筆
        instr=('instr', 'first') if 'instr' in merged_df.columns else ('dummy', 'first')
    )
    .reset_index(drop=True)
)


### 計算成交且穿價時, tmf是long或short的比例

In [71]:
def analyze_directional_fill_ratio(merged_df):
    """
    同時計算：
    - upward cross 時你是 long (sz > 0) 的比例
    - downward cross 時你是 short (sz < 0) 的比例
    """
    df = merged_df.copy()

    # 上穿時成交為 long 的比例
    upward_df = df[df['trade_through'] == 'upward']
    if not upward_df.empty:
        long_ratio = (upward_df['sz'] > 0).mean()
        print(f"在 upward cross 時long TMF 的比例：{long_ratio:.2%}")
    else:
        long_ratio = None

    # 下穿時成交為 short 的比例
    downward_df = df[df['trade_through'] == 'downward']
    if not downward_df.empty:
        short_ratio = (downward_df['sz'] < 0).mean()
        print(f"在 downward cross 時 short TMF 的比例：{short_ratio:.2%}")
    else:
        short_ratio = None

    return None


In [72]:
def analyze_fill_direction_by_trade_through(merged_df):
    df = merged_df.copy()

    # 確保標記為 NaN 的歸類為 no-through
    df['trade_through'] = df['trade_through'].fillna('no-through')

    long_df = df[df['sz'] > 0]
    short_df = df[df['sz'] < 0]

    def compute_ratio(sub_df, label):
        total = len(sub_df)
        if total == 0:
            print(f"\n沒有 {label} TMF 成交")
            return {'upward': None, 'downward': None, 'no-through': None}

        ratio_up = (sub_df['trade_through'] == 'upward').mean()
        ratio_down = (sub_df['trade_through'] == 'downward').mean()
        ratio_none = (sub_df['trade_through'] == 'no-through').mean()

        print(f"\nElectron {label.upper()} TMF 時：")
        print(f"  上穿五檔比例：{ratio_up:.2%}")
        print(f"  下穿五檔比例：{ratio_down:.2%}")
        print(f"  無穿價比例：{ratio_none:.2%}")

        return {
            'upward': ratio_up,
            'downward': ratio_down,
            'no-through': ratio_none
        }

    result = {
        'long_tmfs': compute_ratio(long_df, 'long'),
        'short_tmfs': compute_ratio(short_df, 'short')
    }

    return None

In [73]:
analyze_fill_direction_by_trade_through(merged_sec_df)


Electron LONG TMF 時：
  上穿五檔比例：5.26%
  下穿五檔比例：5.26%
  無穿價比例：89.47%

Electron SHORT TMF 時：
  上穿五檔比例：6.82%
  下穿五檔比例：4.55%
  無穿價比例：88.64%
